# Graph Neutral Network
- **2024/25/1**: Önálló laboratórium 2
- Sági Benedek
- **Konzulens**: Unyi Dániel
- **Projekt és a célja**: A gráf neurális hálózatban a döntésekhez készítünk egy/több magyarázhatósági algoritmust, mely elmondja, hogy egy adott modell miért hozott ilyen döntést.




In [1]:
!pip uninstall torch
!pip install torch==2.4.0

Found existing installation: torch 2.4.0
Uninstalling torch-2.4.0:
  Would remove:
    /usr/local/bin/convert-caffe2-to-onnx
    /usr/local/bin/convert-onnx-to-caffe2
    /usr/local/bin/torchrun
    /usr/local/lib/python3.10/dist-packages/functorch/*
    /usr/local/lib/python3.10/dist-packages/torch-2.4.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/torch/*
    /usr/local/lib/python3.10/dist-packages/torchgen/*
Proceed (Y/n)? y
  Successfully uninstalled torch-2.4.0
  Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torch-2.4.0-cp310-cp310-manylinux1_x86_64.whl (797.2 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.0+cu121 requires torch==2.5.0, but you have torch 2.4.0 which is incompatible.
torchvision 0.20.0+cu121 requires torch==2.5.0, but you have torch 2.4.0 which is incompatible.


In [2]:
!pip install torch_geometric
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.4.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.4.0+cpu.html


In [3]:
import torch

import torch_geometric

from torch_geometric import datasets

from torch_geometric.nn import GCNConv, SAGEConv, global_mean_pool
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.loader import DataLoader

from torch_geometric.loader import NeighborLoader
from torch_geometric.loader import LinkNeighborLoader
from torch_geometric.loader import LinkLoader

import torch_sparse
import torch_geometric.transforms as T

from torch_geometric.utils import negative_sampling
from torch_geometric.utils import train_test_split_edges

import numpy as np

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Lépések:

##1.fázis
---
1. Adatok letöltése, előkészítése
Mivel a pytorch geometrich könyvtárnak van az az API-ja, amivel le lehet tölteni a kiválasztott adathalmazokat.
2. Adatok áttekintése
3. Modellek készítése különböző célokra
  * Csúcs szintű osztályozás
  * Gráf szintű osztályozás
  * Élprédikció

4. Eredmények elemzése
---
Nem ajánlott egyszerre összeset indítani, mivel akkor sokáig fog tartani, főleg a negyediknél.






### 1. Adatok letöltése

In [74]:
#WikipediaNetwork
dataset_WikipediaNetwork = datasets.WikipediaNetwork(root='data/WikipediaNetwork', name = 'chameleon')
#Amazon
dataset_Amazon = datasets.Amazon(root='data/Amazon', name = 'Computers')
#FacebookPagePage
dataset_FacebookPagePage = datasets.FacebookPagePage(root='data/FacebookPagePage')

Processing...
Done!


### 2. Adatok áttekintése

In [100]:
def dataset_info(dataset):
  print(f'Dataset: {dataset}:')
  print('======================')
  print(f'Number of graphs: {len(dataset)}')
  print(f'Number of features: {dataset.num_features}')
  print(f'Number of classes: {dataset.num_classes}')
  data = dataset[0]  # Get the first graph object.
  print(data)
  print(f'Number of nodes: {data.num_nodes}')
  print(f'Number of edges: {data.num_edges}')
  print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
  print(f'Has isolated nodes: {data.has_isolated_nodes()}')
  print(f'Has self-loops: {data.has_self_loops()}')
  print(f'Is undirected: {data.is_undirected()}')

In [102]:
dataset_info(dataset_WikipediaNetwork)
dataset_info(dataset_Amazon)
dataset_info(dataset_FacebookPagePage)

Dataset: WikipediaNetwork():
Number of graphs: 1
Number of features: 2325
Number of classes: 5
Data(x=[2277, 2325], edge_index=[2, 36101], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10])
Number of nodes: 2277
Number of edges: 36101
Average node degree: 15.85
Has isolated nodes: False
Has self-loops: True
Is undirected: False
Dataset: AmazonComputers():
Number of graphs: 1
Number of features: 767
Number of classes: 10
Data(x=[13752, 767], edge_index=[2, 491722], y=[13752])
Number of nodes: 13752
Number of edges: 491722
Average node degree: 35.76
Has isolated nodes: True
Has self-loops: False
Is undirected: True
Dataset: FacebookPagePage():
Number of graphs: 1
Number of features: 128
Number of classes: 4
Data(x=[22470, 128], edge_index=[2, 342004], y=[22470])
Number of nodes: 22470
Number of edges: 342004
Average node degree: 15.22
Has isolated nodes: False
Has self-loops: True
Is undirected: True


#### Következetés
*   Reddit:
Ebben az adathalmazban nincs egyéb érdekesség, azaz indirekt, nincs DAG, és nincsenek izolált pontok. Csak egy gráf.
*   UPFD:
Ám ebben vannak izolált pontok, sőt több gráf is van.
*   Airports:
Itt viszont van DAG.



In [ ]:
for graph_n in range(62):
  data = dataset_UPFD[graph_n]  # Get the first graph object.
  print(graph_n)
  print(data)
  print(f'Number of nodes: {data.num_nodes}')
  print(f'Number of edges: {data.num_edges}')
  print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
  print(f'Has isolated nodes: {data.has_isolated_nodes()}')
  print(f'Has self-loops: {data.has_self_loops()}')
  print(f'Is undirected: {data.is_undirected()}')

### 3. Modellek készítése

#### Csúcs szintű osztályozás
- Gráf konvolucios hálózat
Innen vettem ezt az alapvető kódot: https://colab.research.google.com/drive/14OvFnAXggxB8vM4e8vSURUp1TaKnovzX?usp=sharing#scrollTo=fmXWs1dKIzD8
- Ez alapján bovitettem: https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#convolutional-layers
- A működését ennek segítségével értelmeztem: https://tkipf.github.io/graph-convolutional-networks/

In [97]:
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = SAGEConv(input_dim, hidden_dim)
        self.conv2 = SAGEConv(hidden_dim, hidden_dim)
        self.conv3 = GCNConv(hidden_dim, hidden_dim)
        self.conv4 = GCNConv(hidden_dim, output_dim)

    def  forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.conv3(x, edge_index)
        x = F.relu(x)
        x = self.conv4(x, edge_index)
        return F.log_softmax(x, dim=1)

#### Élprédikáció

In [98]:
class LinkPredictor(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(LinkPredictor, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim * 2, 1)

    def encode(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        return self.conv2(x, edge_index)

    def decode(self, z, edge_index):
        src, dst = edge_index
        z_src, z_dst = z[src], z[dst]
        return torch.sigmoid(self.fc(torch.cat([z_src, z_dst], dim=1)))

    def forward(self, data):
        z = self.encode(data.x, data.edge_index)
        return self.decode(z, data.edge_index)

#### Gráfszintu osztalyozas

In [99]:
class GCNGraphClassifier(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNGraphClassifier, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, hidden_dim)
        self.fc = torch.nn.Linear(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        x = global_mean_pool(x, batch)  # Gráf szintű pooling
        return F.log_softmax(self.fc(x), dim=1)


### 4. Eredmények és elemzése

In [25]:
batch_size = 128
num_neighbors = [10,10]
lr = 1e-3
hidden_dim = 64

In [54]:
def creatingMask(dataset, train_ratio=0.8, test_ratio = 0.2):
  data = dataset[0]
  num_nodes = data.num_nodes
  perm = torch.randperm(data.num_nodes)
  train_mask = perm[:int(data.num_nodes * train_ratio)]
  test_mask = perm[int(data.num_nodes * test_ratio):]
  data.train_mask = torch.zeros(num_nodes, dtype=torch.bool)
  data.test_mask = torch.zeros(num_nodes, dtype=torch.bool)
  data.train_mask[train_mask] = True
  data.test_mask[test_mask] = True
  return data

#### Csúcs szintű osztályozás

In [47]:
def node_train_model(train_loader, test_loader, optimizer, model, device):
  def train():
      count=0
      model.train()
      total_loss = 0
      for batch in train_loader:
          batch = batch.to(device)
          optimizer.zero_grad()
          out = model(batch.x, batch.edge_index)
          criterion = CrossEntropyLoss()
          loss = criterion(out[:batch.batch_size], batch.y[:batch.batch_size])
          loss.backward()
          optimizer.step()
          total_loss += loss.item()
      return total_loss / len(train_loader)
  def test():
      model.eval()
      correct = 0
      total = 0  # Összes tesztadat száma
      for batch in test_loader:
          batch = batch.to(device)
          out = model(batch.x, batch.edge_index)
          pred = out.max(dim=1)[1]
          correct += pred.eq(batch.y).sum().item() # Kiszámolja, hogy abbol mennyi el lett találva az eq metódus segítségével
          total += batch.y.size(0)  # Összes minta száma
      return correct / total
  for epoch in range(10):
    loss = train()
    acc = test()
    #Elemzés
    print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')

In [67]:
def nodeMainModel(data, num_classes, num_features):
  train_loader = NeighborLoader(data, num_neighbors=num_neighbors , batch_size=batch_size, input_nodes=data.train_mask)
  test_loader = NeighborLoader(data, num_neighbors=num_neighbors , batch_size=batch_size, input_nodes=data.test_mask)
  model = GCN(input_dim=num_features, hidden_dim=hidden_dim, output_dim=num_classes).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=5e-4)
  node_train_model(train_loader, test_loader, optimizer, model, device)

In [94]:
dataset_WikipediaNetwork[0]

Data(x=[2277, 2325], edge_index=[2, 36101], y=[2277], train_mask=[2277, 10], val_mask=[2277, 10], test_mask=[2277, 10])

In [96]:
#with masks
for dataset in [dataset_WikipediaNetwork]:
  print(dataset)
  nodeMainModel(dataset[0],dataset.num_classes, dataset.num_features)
#without masks
for dataset in [dataset_Amazon,dataset_FacebookPagePage]:
  print(dataset)
  nodeMainModel(creatingMask(dataset), dataset.num_classes, dataset.num_features)

WikipediaNetwork()


RuntimeError: view size is not compatible with input tensor's size and stride (at least one dimension spans across two contiguous subspaces). Use .reshape(...) instead.

#### Élprédikáció

In [46]:
def edge_train_model(train_loader, test_loader, optimizer, model, device):
    def train():
        total_loss = 0
        model.train()
        for batch in train_loader:
            batch = batch.to(device)
            optimizer.zero_grad()

            # Kódolás
            z = model.encode(batch.x, batch.edge_index)

            # Pozitív élek dekódolása
            pos_pred = model.decode(z, batch.edge_index)

            # Negatív minták generálása
            neg_edge_index = negative_sampling(
                edge_index=batch.edge_index, num_nodes=batch.x.size(0), num_neg_samples=batch.edge_index.size(1)
            )

            # Negatív élek dekódolása
            neg_pred = model.decode(z, neg_edge_index)

            # Loss számítása: Pozitív és negatív minták összehasonlítása
            pos_loss = F.binary_cross_entropy(pos_pred.squeeze(), torch.ones(pos_pred.size(0)).to(device))
            neg_loss = F.binary_cross_entropy(neg_pred.squeeze(), torch.zeros(neg_pred.size(0)).to(device))

            # Teljes veszteség
            loss = pos_loss + neg_loss
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
        return total_loss / len(train_loader)

    def test():
        model.eval()
        correct = 0
        total = 0  # Összes tesztadat száma

        with torch.no_grad():
            for batch in test_loader:
                batch = batch.to(device)

                # Kódolás és predikció
                z = model.encode(batch.x, batch.edge_index)

                # Pozitív élek predikciója
                pos_pred = model.decode(z, batch.edge_index)
                '''
                # Negatív élek mintavételezése (azaz BSZ2 szempontjából komplementere)
                neg_edge_index = negative_sampling(
                    edge_index=batch.edge_index, num_nodes=batch.x.size(0), num_neg_samples=batch.edge_index.size(1)
                )
                neg_pred = model.decode(z, neg_edge_index)
                '''
                # Predikciók összehasonlítása
                pos_correct = (pos_pred > 0.5).sum().item()
                #neg_correct = (neg_pred < 0.5).sum().item()

                #correct += pos_correct + neg_correct
                correct += pos_correct
                #total += pos_pred.size(0) + neg_pred.size(0)
                total += pos_pred.size(0)
        return correct / total

    for epoch in range(10):
        loss = train()
        acc = test()
        # Elemzés
        print(f'Epoch {epoch}, Loss: {loss:.4f}, Test Accuracy: {acc:.4f}')


In [ ]:
def edgeMainModel(data,  num_classes, num_features):

  transform = RandomLinkSplit(is_undirected=True)
  train_data, val_data, test_data = transform(data)

  train_loader = LinkNeighborLoader(
      train_data,
      num_neighbors=num_neighbors,
      batch_size=batch_size,
      edge_label_index=train_data.edge_index,
      shuffle=True
  )

  test_loader = LinkNeighborLoader(
      test_data,
      num_neighbors=num_neighbors,
      batch_size=batch_size,
      edge_label_index=test_data.edge_index,
      shuffle=False
  )

  model = LinkPredictor(num_node_features, hidden_dim=hidden_dim).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=lr)
  edge_train_model(train_loader, test_loader, optimizer, model, device)

In [ ]:
#with masks
for dataset in [dataset_Reddit]:
  print(dataset)
  edgeMainModel(dataset[0])
#without masks
for dataset in [dataset_FacebookPagePage]:
  print(dataset)
  edgeMainModel(creatingMask(dataset[0]))

#### Gráfszintű osztályozás

In [ ]:
def datasetWithMask(dataset, train_ratio, test_ratio, model):
  data = dataset[0]

model = GCNGraphClassifier(data.num_edge_features, hidden_dim, data.num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## 2.fázis
--------
Folytatás
--------
